## Base de Datos Original

In [5]:
import pandas as pd

La base de datos sobre la que partira el estudio sera aquella en la que las paradas tanto de transacciones como de la base de paradas coincidan.

In [6]:
smart_data = pd.read_csv('smart_data_filtrado_paradas_coincidentes.csv', sep = ',')
smart_data["FechaHoraValidacion"] = pd.to_datetime(smart_data["FechaHoraValidacion"])

## Particiones y Estadisticas

Observamos la cantidad de registros que existen por numero de transacciones diarias.

In [8]:
# Contar la cantidad de registros por Fecha y CodigoTarjeta
conteo = smart_data.groupby(['Fecha', 'CodigoTarjeta']).size().reset_index(name='Cantidad')

# Filtrar los que tienen 3 transacciones diarias
validos = conteo[conteo['Cantidad'] == 5 ]

# Hacer merge para conservar solo esos registros en el DataFrame original
smart_data_de_3 = smart_data.merge(validos[['Fecha', 'CodigoTarjeta']], on=['Fecha', 'CodigoTarjeta'], how='inner')

# Contar la cantidad total de transacciones aportadas por estos usuarios
total_transacciones = len(smart_data_de_3)

print(f'Total de transacciones de usuarios con 5 viajes diarios: {total_transacciones}')


Total de transacciones de usuarios con 5 viajes diarios: 629035


In [20]:
import pandas as pd

# Asegurar tipo datetime
smart_data['FechaHoraValidacion'] = pd.to_datetime(smart_data['FechaHoraValidacion'])

# Contar transacciones por Fecha y CodigoTarjeta
conteo = smart_data.groupby(['Fecha', 'CodigoTarjeta']).size().reset_index(name='Cantidad')

# Filtrar los que tienen exactamente 2
exactos_dos = conteo[conteo['Cantidad'] == 2]

# Filtrar el DataFrame original
data_dos = smart_data.merge(exactos_dos[['Fecha', 'CodigoTarjeta']], on=['Fecha', 'CodigoTarjeta'], how='inner')

# Ordenar por tarjeta, fecha y hora
data_dos = data_dos.sort_values(by=['CodigoTarjeta', 'Fecha', 'FechaHoraValidacion'])

# Agregar índice para numerar los registros (0 para primero, 1 para segundo)
data_dos['pos'] = data_dos.groupby(['Fecha', 'CodigoTarjeta']).cumcount()

# Separar los dos registros
primero = data_dos[data_dos['pos'] == 0].copy()
segundo = data_dos[data_dos['pos'] == 1].copy()

# Calcular diferencia de tiempo
tiempo_dif = segundo['FechaHoraValidacion'].values - primero['FechaHoraValidacion'].values

# Seleccionar solo los pares con más de 2 horas de diferencia
mask_valida = tiempo_dif < pd.Timedelta(hours=2)

# Combinar ambos registros válidos en un solo DataFrame
resultado = pd.concat([
    primero[mask_valida],
    segundo[mask_valida]
])

# Ordenar resultado final
resultado = resultado.sort_values(by=['CodigoTarjeta', 'Fecha', 'FechaHoraValidacion'])

In [27]:
resultado.to_csv('NO_DETERMINADOS/smart_dos_transacciones_menos2_horas.csv', index = False)

## Usuarios con 1 Solo Registro y Promedio Semanal de 1 

Filtramos a los usuarios con una sola transaccion al dia para inferir despues.

In [19]:
# Contar la cantidad de registros por Fecha y CodigoTarjeta
conteo = smart_data.groupby(['Fecha', 'CodigoTarjeta']).size().reset_index(name='Cantidad')
# Filtrar solo los que tienen 1 registro
validos = conteo[conteo['Cantidad'] == 1]
# Hacer merge para conservar solo esos registros en el DataFrame original
smart_data_filtrado = smart_data.merge(validos[['Fecha', 'CodigoTarjeta']], on=['Fecha', 'CodigoTarjeta'], how='inner')

In [20]:
smart_data_filtrado.to_csv('NO_DETERMINADOS/smart_una_transaccion_dia.csv', index = False)

Seleccionamos a los usuarios con almenos 2 transacciones por dia y menos de 6 para realizar la inferencia.

In [21]:
# Contar la cantidad de registros por Fecha y CodigoTarjeta
conteo = smart_data.groupby(['Fecha', 'CodigoTarjeta']).size().reset_index(name='Cantidad')

# Filtrar solo los que tienen entre 2 y 6 transacciones (inclusive)
validos = conteo[(conteo['Cantidad'] >= 2) & (conteo['Cantidad'] <= 6)]

# Hacer merge para conservar solo esos registros en el DataFrame original
smart_data_filtrado = smart_data.merge(validos[['Fecha', 'CodigoTarjeta']], on=['Fecha', 'CodigoTarjeta'], how='inner')


In [17]:
smart_data_filtrado.to_csv('DETERMINADOS/smart_dos_a_seis_transacciones.csv', index = False)

Filtramos a los usuarios con un solo dia de registro por semana.

In [12]:
smart_data['Fecha'] = smart_data['FechaHoraValidacion'].dt.date
smart_data['Semana'] = smart_data['FechaHoraValidacion'].dt.isocalendar().week
transacciones = smart_data.groupby(['CodigoTarjeta', 'Semana', 'Fecha']).size().reset_index(name='TransaccionesDia')
semana_usuario = transacciones.groupby(['CodigoTarjeta', 'Semana']).agg(
    DiasActivosSemana=('Fecha', 'nunique'),
    PromedioPorDiaSemana=('TransaccionesDia', 'sum')
).reset_index()
resumen_mensual = semana_usuario.groupby('CodigoTarjeta').agg(
    PromedioDiasActivos=('DiasActivosSemana', 'mean'),
    TotalTransaccionesMes=('PromedioPorDiaSemana', 'sum')
).reset_index()

## Filtrado Regulares e Irregulares

In [1]:
import pandas as pd
import ast
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#------------------------BASE DE DATOS CON LOS REGISTROS INFERIDOS---------------------------------------
smart_inferidos = pd.read_csv('base_total_DL.csv', sep = ',')
smart_inferidos = smart_inferidos.drop(columns=['ParadaRegreso', 'FechaHoraRegreso'])
smart_inferidos = smart_inferidos.rename(columns={'ParadaAscenso_Nombre': 'Origen'})
# Obtener la lista de IDs únicos en la columna 'Origen'
ids_origen = smart_inferidos['Origen'].unique()
# Filtrar las filas donde 'Destino' está en la lista de 'Origen'
smart_filtrado = smart_inferidos[smart_inferidos['Destino'].isin(ids_origen)]

In [3]:
#------------------------BASES DE REGULARES E IRREGULARES-------------------------------------------------
smart_regulares = pd.read_csv('smart_regulares.csv', sep = ',')
smart_irregulares = pd.read_csv('smart_irregulares.csv', sep = ',')


In [4]:
#Entrenar dos modelos tanto para regulares como irregulares
smart_inferidos_regulares = smart_inferidos[smart_inferidos['CodigoTarjeta'].isin(smart_regulares['CodigoTarjeta'])]
smart_inferidos_irregulares = smart_inferidos[smart_inferidos['CodigoTarjeta'].isin(smart_irregulares['CodigoTarjeta'])]

In [5]:
#------------------------BASES NO DETERMINADAS-----------------------------------------------------------
smart_inferir = pd.read_csv('NO_DETERMINADOS/base_a_predecir_definitiva.csv', sep = ',')

In [6]:
#Predecir con los modelos correspondientes tanto para regulares como irregulares
smart_predecir_regulares = smart_inferir[smart_inferir['CodigoTarjeta'].isin(smart_regulares['CodigoTarjeta'])]
smart_predecir_irregulares = smart_inferir[smart_inferir['CodigoTarjeta'].isin(smart_irregulares['CodigoTarjeta'])]

In [7]:
#------------------------GUARDAMOS LAS BASES DE DATOS-----------------------
smart_inferidos_regulares.to_csv("BASES_FINALES_DL/smart_inferidos_regulares.csv", index = False)
smart_inferidos_irregulares.to_csv("BASES_FINALES_DL/smart_inferidos_irregulares.csv", index = False)
smart_predecir_regulares.to_csv("BASES_FINALES_DL/smart_predecir_regulares.csv", index = False)
smart_predecir_irregulares.to_csv("BASES_FINALES_DL/smart_predecir_irregulares.csv", index = False)